In [1]:
import numpy as np

step1: 変数

1.2 Variableクラスの実装

In [2]:
class Variable:
    def __init__(self, data):
        self.data = data

In [3]:
data = np.array(1.0)
x = Variable(data)
print(x.data)

x.data = 2.0
print(x.data)

1.0
2.0


1.3 Numpyの多次元配列

In [4]:
x = np.array(1)
print(x.ndim)

y = np.array([1, 2, 3])
print(y.ndim)

z = np.array([[1, 2, 3],
              [4, 5, 6]])
print(z.ndim)

0
1
2


step2: 変数を生み出す関数

2.2 Functionクラスの実装

In [5]:
class Function:
    # f = Function()とした時, f(...)で__call__を呼べる
    def __call__(self, input): # Variableインスタンス
        x = input.data # Valuableインスタンス内のデータ
        y = x ** 2 # xをyに変換
        output = Variable(y) # 新しくVariableインスタンス生成しyをデータとして格納
        return output # yを格納したインスタンスを返す

2.3 Functionクラスを使う

In [6]:
x = Variable(np.array(10))
f = Function()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [7]:
class Function:
    # f = Functionとした時, f(...)で__call__を呼べる
    def __call__(self, input):
        x = input.data
        # y = x ** 2の代わり
        y = self.forward(x) # 具体的計算は別クラスが搭載するforwardメソットでする
        output = Variable(y)
        return output

    def forward(self, x):
        # 他のクラスでFunctionクラスを引き継いでforwardを再定義する
        raise NotImplementedError() # Functionインスタンスからforwardを呼ぶと例外が発生

In [8]:
class Square(Function): # Functionクラスを引き継ぐ
    # forwardを再定義
    def forward(self, x):
        return x ** 2

In [9]:
x = Variable(np.array(10))
f = Square() # f = Function()
y = f(x) # この時点で__call__が呼ばれる(その中にあるforwardも呼ばれて計算が行われる)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


step3: 関数の連結

3.1 Exp関数の実装

In [10]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

3.2 関数を連結する

In [11]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
# 入力Variableインスタンスのデータを取り出して計算後、その計算結果を新しいVariableインスタンスのデータに格納、の繰り返し
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


step4: 数値微分

4.2 数値微分の実装

In [12]:
def numerical_diff(f, x, eps=1e-4): # 扱う関数、Variableインスタンス
    x0 = Variable(x.data - eps) # インスタンスのデータに少し変化加えて新しいVariableインスタンスのデータに格納
    x1 = Variable(x.data + eps) # 同様
    y0 = f(x0) # fのforwardに通して計算され、新しいVariableインスタンスのデータに格納
    y1 = f(x1) # 同様
    return (y1.data - y0.data) / (2 * eps) # 各インスタンスのデータを呼んで数値微分し返す

In [13]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x) # 関数fに対してxにおける数値微分
print(dy)

4.000000000004


4.3 合成関数の数値微分

In [14]:
# 連なった関数（複数の関数計算を一つの関数としてまとめる）での数値微分
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
dx = numerical_diff(f, x)
print(dx) # xを0.5から微小な値だけ変化させた時、yの値がその微小な変化のdy倍だけ変化する

3.2974426293330694


step5,6: 手動逆伝播

6.1 Variableクラスの追加実装

In [15]:
class Variable:

    def __init__(self, data):
        self.data = data
        self.grad = None # 逆伝播の勾配ここに格納（変数の数だけ勾配があるから、それぞれの変数に対応する勾配を格納させる）

6.2 Functionクラスの追加実装

In [16]:
class Function:

    def __call__(self, input): # Variableインスタンス（データを保持）
        self.input = input # 入力Variableインスタンスを保持(逆伝播のために)
        x = input.data # 入力Variableのデータ
        y = self.forward(x) # 前伝播
        output = Variable(y) # 前伝播の出力データを新しいVariableインスタンスに格納
        return output

    def forward(self, x):
        # 他のクラスでFunctionクラスを引き継いでforwardを再定義する
        raise NotImplementedError

    def backward(self, gy):
        # 他のクラスでFunctionクラスを引き継いでbackwardpを再定義する
        raise NotImplementedError

6.3 SquareとExpクラスの追加実装

In [17]:
class Square(Function):

    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data # Functionに入力Variableインスタンスが保持されていてそれを引き継ぐ
        gx = 2 * x * gy
        return gx

In [18]:
class Exp(Function):

    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data # Functionに入力Variableインスタンスが保持されていてそれを引き継ぐ
        gx = np.exp(x) * gy
        return gx

6.4 バックプロパゲーションの実装

In [19]:
A = Square()
B = Exp()
C = Square()

# 前伝播
x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 逆伝播（dx以外の勾配もついでに求められる）
y.grad = np.array(1.0) # 学伝播は１から始まる
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)


# 数値微分との比較（数値微分はdxしか求められていない）
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
dx = numerical_diff(f, x)
print(dx)


3.297442541400256
3.2974426293330694


step7: 逆伝播の自動化

7.1 逆伝播の自動化のために親（関数）と子（入力/出力）を関連付ける

In [20]:
class Variable:

    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None # 自分（変数）を生み出した親（関数）の情報を保持

    def set_creator(self, func): # この変数を生み出した関数
        self.creator = func

In [21]:
class Function:
    def __call__(self, input):
        self.input = input # 関数(親)に入力（前の親の子）データを保持させる（逆伝播のため）
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self) # output（子）内にself(使用した関数)をcreator（親）として格納
        self.output = output # 関数(親)に出力（子）データを保持させる
        return output

In [22]:
class Square(Function):

    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [23]:
class Exp(Function):

    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [24]:
A = Square()
B = Exp()
C = Square()

# 前伝播
x = Variable(np.array(0.5))
# この計算中、出力（子）は自分を生み出した関数（親）を、関数(親は)は出力（子）と入力（前の親の子）を保持する
a = A(x)
b = B(a)
y = C(b)

# 一つずつの親と子を確認
assert y.creator == C
assert y.creator.input ==b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

7.2 関連付けた親と子の逆伝播を試す(手動)


In [25]:
y.grad = np.array(1.0)

C = y.creator # 最後の出力を出した関数（親）を取得
b = C.input # その関数（親）が保持する入力（Variableインスタンス）を取得
b.grad = C.backward(y.grad) # 入力Variableインスタンスに逆伝播で勾配を渡す

B = b.creator
a = B.input
a.grad = B.backward(b.grad)

A = a.creator
x = A.input
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


7.3 backwardメソッドをVariableクラスに追加し7.2を自動化する（再帰：途中の結果をメモリに残してしまう）

In [26]:
class Variable:

    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    # 上の逆伝播のフローをVariableインスタンス内で自動化
    def backward(self):
        f = self.creator # 出力Variableインスタンスを生み出した関数を取得
        if f is not None: # このインスタンスに生みの親（関数）がいるか（一番最初の入力は生みの親（関数）がない）
            x = f.input # その関数の入力Variableインスタンスを取得
            x.grad = f.backward(self.grad) # その入力に対応する勾配を関数の逆伝播計算からを求める（引数は一つ前の逆伝播（出力）から渡された勾配）
            x.backward() # 同様のことを入力インスタンスでも逆伝播を行い最初の入力まで繰り返しバトンタッチする（再帰)

In [27]:
A = Square()
B = Exp()
C = Square()

# 前伝播
x = Variable(np.array(0.5))
# この計算中、出力（子）は自分を生み出した関数（親）を、関数(親は)は出力（子）と入力（別の親の子）を保持する
a = A(x)
b = B(a)
y = C(b)

# 逆伝播
y.grad = np.array(1.0)
y.backward() # 入力まで自動で逆伝播を行う
print(x.grad)

3.297442541400256


step8: 再起からループへ（メモリ効率向上や拡張性：step15でその恩恵がわかる）

8.2 ループを使った実装（途中の結果をメモリに残さない）

In [28]:
class Variable:

    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func # 前伝播する際に取得

    def backward(self):
        funcs = [self.creator] # 出力の親をリストに格納
        while funcs: # 直線的な計算グラフなら通常一つの関数、分岐する計算グラフなら複数の関数を含む
            f = funcs.pop() # リストの末尾の要素を取り出す（この時リストからその要素は除外）
            x, y = f.input, f.output # その関数の入力と出力を取得し
            x.grad = f.backward(y.grad) # 入力xにに対する勾配を求める

            if x.creator is not None: # 入力に親がいるならそれをリストに追加してループ（最初の入力は親（関数）をもたない）
                funcs.append(x.creator)

In [29]:
A = Square()
B = Exp()
C = Square()

# 前伝播
x = Variable(np.array(0.5))
# この計算中、出力（子）は自分を生み出した関数（親）を、関数(親は)は出力（子）と入力（別の親の子）を保持する
a = A(x)
b = B(a)
y = C(b)

# 逆伝播
y.grad = np.array(1.0)
y.backward() # 入力まで自動で逆伝播を行う
print(x.grad)

3.297442541400256


step9: 関数をより便利に

9.1 関数呼び出しの簡略化

In [30]:
x = Variable(np.array(0.5))

f = Square()
y = y = f(x)
# 関数計算に２行必要

In [31]:
import numpy as np
# 一度上の２行を一つの関数に定義すれば次から１行で済む
def square(x):
    f = Square()
    return f(x) # Square() (x)に１行にまとめることも可能

In [32]:
def exp(x):
    f = Exp()
    return f(x) # Exp() (x)に１行にまとめることも可能

In [33]:
# 前伝播
x = Variable(np.array(0.5))
# この計算中、出力（子）は自分を生み出した関数（親）を、関数(親は)は出力（子）と入力（別の親の子）を保持する
# a = square(x)
# b = exp(a)
# y = square(b)
y = square(exp(square(x)))

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


9.2 backwardメソッドの簡略化

In [34]:
class Variable:

    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None: # 逆伝播の最初の計算(最終出力)には、渡される勾配がない(１で固定)
            self.grad = np.ones_like(self.data) # 最終出力と同じ形状の１を生成しそれを渡された勾配とする

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [35]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.backward()
print(x.grad)

3.297442541400256


9.3 ndarrayだけを扱う

In [36]:
class Variable:

    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray): # dataがndarrayインスタンスではない時
                raise TypeError('{} is not supported'.format(type(data))) # 入力されたdataのデータ型を{}に表示

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [37]:
x = Variable(np.array(1.0))
x = Variable(None)

# x = Variable(1.0) # ndarrayとNone以外はエラーが表示される

In [38]:
x = np.array([1.0]) # 1次元
y = x ** 2 # 1次元のndaarayを保つ
print(x, type(x), x.ndim)
print(y, type(y), y.ndim)

[1.] <class 'numpy.ndarray'> 1
[1.] <class 'numpy.ndarray'> 1


In [39]:
x = np.array(1.0) # ０次元（一応ndarrayだが実質スカラ）
y = x ** 2 # ndarrayの型を失ってスカラ系の型（floatなど）になる
print(x, type(x), x.ndim)
print(y, type(y), y.ndim)

1.0 <class 'numpy.ndarray'> 0
1.0 <class 'numpy.float64'> 0


In [40]:
def as_array(x):
    if np.isscalar(x): # もしfloatなどのスカラ系の型なら
        return np.array(x) # 改めて型をndarrayにする
    return x

In [41]:
x = np.array(1.0) # ０次元（一応ndarrayだが実質スカラ）
y = x ** 2 # ndarrayの型を失ってスカラ系の型（floatなど）になる
y = as_array(y) # 出力がスカラになった場合ndarrayに変換（元々ndarrayならそのまま帰す）
print(x, type(x), x.ndim)
print(y, type(y), y.ndim)

1.0 <class 'numpy.ndarray'> 0
1.0 <class 'numpy.ndarray'> 0


In [42]:
class Function:
    def __call__(self, input):
        self.input = input
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y)) # 出力がスカラになった場合ndarrayに変換（元々ndarrayならそのまま返す）して新しいVaribaleインスタンスのデータ二格納
        output.set_creator(self)
        self.output = output
        return output

In [43]:
class Square(Function):

    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [44]:
class Exp(Function):

    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

step10: テスト

10.1, 10.2 Pythonのユニットテスト(square関数の前伝播と逆伝播)

In [45]:
import unittest

class SquareTest(unittest.TestCase):

    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

unittest.main(argv=[""], verbosity=2, exit=False)

test_forward (__main__.SquareTest.test_forward) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [46]:
import unittest

class SquareTest(unittest.TestCase):

    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

unittest.main(argv=[""], verbosity=2, exit=False)

test_backward (__main__.SquareTest.test_backward) ... ok
test_forward (__main__.SquareTest.test_forward) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK


10.3 勾配確認による自動テスト

In [47]:
def numerical_diff(f, x, eps=1e-4): # 扱う関数、Variableインスタンス
    x0 = Variable(x.data - eps) # インスタンスのデータに少し変化加えて新しいVariableインスタンスのデータに格納
    x1 = Variable(x.data + eps) # 同様
    y0 = f(x0) # fのforwardに通して計算され、新しいVariableインスタンスのデータに格納
    y1 = f(x1) # 同様
    return (y1.data - y0.data) / (2 * eps) # 各インスタンスのデータを呼んで数値微分し返す

class SquareTest(unittest.TestCase):

    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)

    def test_gradient_check(self):
        x = Variable(np.random.rand(1)) # ランダムな入力値を生成
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)

unittest.main(argv=[""], verbosity=2, exit=False)

test_backward (__main__.SquareTest.test_backward) ... ok
test_forward (__main__.SquareTest.test_forward) ... ok
test_gradient_check (__main__.SquareTest.test_gradient_check) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK
